In [ ]:
import obi_auth
import pandas as pd
from entitysdk import Client
from ipywidgets import widgets
from obi_notebook import get_entities, get_projects
from obi_one.scientific.library import circuit_metrics, connectivity_metrics, circuit
import obi_one as obi
environment = 'staging'

In [ ]:
token = obi_auth.get_token(environment=environment)

In [ ]:
project_context = get_projects.get_projects(token, env=environment)

In [ ]:
circuit_ids = get_entities.get_entities(
    "circuit",
    token,
    result=[],
    return_entities=False,
    multi_select=False,
    show_pages=True,
    project_context=project_context,
    page_size=10,
    default_scale="small",
    exclude_scales=["single"],
    add_columns=["subject.name"],
    env=environment
)

In [ ]:
# Retrieve name of edge populations (as part of circuit metrics)
db_client = Client(environment=project_context.environment, project_context=project_context, token_manager=token)
metrics = circuit_metrics.get_circuit_metrics(circuit_id=circuit_ids[0], db_client=db_client)

In [ ]:
e_popul_wdgt = widgets.Dropdown(options=metrics.names_of_chemical_edge_populations, description="Edge population:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(e_popul_wdgt)

In [ ]:
# Select group-by property
# --> Valid options to be loaded from the circuit
# --> Depends on the pre-/post-synaptic node populations of the selected edges
group_by = "mtype"  # None, "layer", "mtype", ...

# Select pre-/post-synaptic neuron selections
# --> Valid options to be retrieved from the circuit
# --> Depends on the pre-/post-synaptic node populations of the selected edges
pre_selection = {"etype": ["bAC", "cNAC"]}
pre_node_set = None
post_selection = {"layer": "SP"}
post_node_set = "Inhibitory"

# Select max. distance to take connections into account
max_distance = None  # None, 100, 1000, ... (um)

In [ ]:
conn_output = connectivity_metrics.get_connectivity_metrics(
    circuit_id=circuit_ids[0],
    db_client=db_client,
    edge_population=e_popul_wdgt.value,
    pre_selection=pre_selection,
    pre_node_set=pre_node_set,
    post_selection=post_selection,
    post_node_set=post_node_set,
    group_by=group_by,
    max_distance=max_distance
)

In [ ]:
# Convert connection probability output to a dataframe
pd.DataFrame(conn_output.connection_probability)

In [ ]:
# Convert #synapses per connection output to a dataframe
pd.DataFrame(conn_output.mean_number_of_synapses)

In [ ]:
circuit_id = circuit_ids[0]

In [ ]:
# This cell shows how to call the connectivity-metrics API endpoint directly
# First launch the OBI-ONE API locally (see instructions in the OBI-ONE repo)
# The example works with the example input parameters defined below
# But these can be changed as needed

import requests

circuit_id = "d7344f4b-422a-4484-9cc4-794dbe0cf388"
edge_population = "CA3_projections__hippocampus_neurons__chemical_synapse"
group_by = "mtype"  # None, "layer", "mtype", ...

# Select pre-/post-synaptic neuron selections
pre_selection = None
pre_node_set = None
post_selection = None
post_node_set = None

# Select max. distance to take connections into account
max_distance = None  # None, 100, 1000, ... (um)

obi_one_api_url = "http://127.0.0.1:8100"
url = f"{obi_one_api_url}/declared/connectivity-metrics"

params = {
    "circuit_id": circuit_id,
    "edge_population": edge_population,
    "pre_selection": pre_selection,
    "pre_node_set": pre_node_set,
    "post_selection": post_selection,
    "post_node_set": post_node_set,
    "group_by": group_by,
    "max_distance": max_distance,
}

headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
}

headers["virtual-lab-id"] = obi.LAB_ID_STAGING_TEST
headers["project-id"] = obi.PROJECT_ID_STAGING_TEST

response = requests.post(url, headers=headers, json=params)

if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")